# Token-Based Authentication with JWT and OAuth2 in FastAPI

In this tutorial, we'll delve into how to implement token-based authentication in a FastAPI application using JSON Web Tokens (JWT)) and OAuth2. We'll cover the fundamental concepts, explain the key components, and provide step-by-step instructions with code examples. By the end of this tutorial, you'll have a solid understanding of how to secure your FastAPI applications using JWT and OAuth2.

## Table of Contents

1. [Introduction](#1-introduction)
2. [Prerequisites](#2-prerequisites)
3. [Understanding Authentication and Authorization](#3-understanding-authentication-and-authorization)
4. [What is JWT?](#4-what-is-jwt)
5. [Understanding OAuth2](#5-understanding-oauth2)
6. [Setting Up the Environment](#6-setting-up-the-environment)
7. [Installing Dependencies](#7-installing-dependencies)
8. [Creating the FastAPI Application](#8-creating-the-fastapi-application)
9. [Defining the User Model](#9-defining-the-user-model)
10. [Creating Pydantic Schemas](#10-creating-pydantic-schemas)
11. [Password Hashing with Passlib](#11-password-hashing-with-passlib)
12. [Implementing JWT Token Generation](#12-implementing-jwt-token-generation)
13. [Setting Up OAuth2 with Password Flow](#13-setting-up-oauth2-with-password-flow)
14. [Securing Endpoints](#14-securing-endpoints)
15. [Testing the Authentication Flow](#15-testing-the-authentication-flow)
16. [Advanced Topics](#16-advanced-topics)
    - [16.1. Token Refresh](#161-token-refresh)
    - [16.2. Revoking Tokens](#162-revoking-tokens)
    - [16.3. Role-Based Access Control (RBAC)](#163-role-based-access-control-rbac)
17. [Conclusion](#17-conclusion)
18. [References](#18-references)

## 1. Introduction

Authentication and authorization are critical components of modern web applications. FastAPI, a high-performance web) web framework for building APIs with Python 3.6+, provides robust support for authentication using OAuth2 and JWT.

In this tutorial, we'll:

- Understand the concepts of authentication and authorization.
- Learn about JWT and its role in stateless authentication.
- Implement OAuth2 password flow in FastAPI.
- Secure API endpoints with JWT tokens.
- Explore advanced topics like token refresh and role-based access control.

## 2. Prerequisites

Before we begin, ensure you have the following:

- **Python 3.7+** installed.
- Basic knowledge of **Python**, **FastAPI**, and **asynchronous programming**.
- Familiarity with **OAuth2** and **JWT** concepts is helpful but not required.

## 3. Understanding Authentication and Authorization

- ****Authentication**: The process of verifying who a user is.
- **Authorization**: The process of verifying what they have access to.

In the context of APIs, authentication confirms the identity of a user or service, and authorization determines their permissions.

## 4. What is JWT?

**JSON Web Token (JWT)** is an open standard for securely transmitting information between parties as a JSON object. It's widely used for authentication and information exchange.

### 4.1. Structure of JWT

A JWT is composed of three parts:

1. **Header**: Specifies the algorithm used for signing (e.g., HS256).
2. **Payload**: Contains the claims (user data and metadata).
3. **Signature**: Used to verify the token's integrity.

Encoded as:

```
Header.Payload.Signature
```

## 5. Understanding OAuth2

**OAuth2** is an authorization framework that enables applications to obtain limited access to user accounts on an HTTP service.

### 5.1. OAuth2 Flows

- **Authorization Code**: Used with server-side applications.
- **Implicit**: Used with mobile or web applications.
- **Resource Owner Password Credentials**: Direct exchange of username and password for a token.
- **Client Credentials**: Machine-to-machine authentication.

In this tutorial, we'll use the **Resource Owner Password Credentials** flow.

## 6. Setting Up the Environment

Create a new project directory and set up a virtual environment.

```bash
# Create project directory
mkdir fastapi-jwt-auth
cd fastapi-jwt-auth

# Set up virtual environment
python -m venv venv
source venv/bin/activate  # On Windows: venv\Scripts\activate
```

## 7. Installing Dependencies

Install the necessary packages.

```bash
pip install fastapi uvicorn[standard] passlib[bcrypt] python-jose[cryptography] python-multipart
```

- **fastapi**: The web framework.
- **uvicorn**: ASGI server.
- **passlib[bcrypt]**: For secure password hashing.
- **python-jose[cryptography]**: For JWT encoding and decoding.
- **python-multipart**: For form data parsing.

## 8. Creating the FastAPI Application

**Directory Structure**

```
fastapi-jwt-auth/
├── main.py
├── models.py
├── schemas.py
├── auth.py
├── database.py
``── utils.py
``── requirements.txt
``````

Create the files:

```bash
touch main.py models.py schemas.py auth.py database.py utils.py
```

`requirements.txt` will be generated automatically when you freeze dependencies.

## 9. Defining the User Model

We'll simulate a database with an in-memory users' dictionary for simplicity.

**models.py**

```python
from typing import Optional
from passlib.context import CryptContext

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="="auto")

# Simulated database
fake_users_db = {
    "alice": {
        "username": "alice",
        "full_name": "Alice Wonderland",
        "email": "alice@example.com",
        "hashed_password": pwd_context.hash("secret"),
        "disabled": False,
    },
    "bob": {
        "username": "bob",
        "full_name": "Bob Builder",
        "email": "bob@example.com",
        "hashed_password": pwd_context.hash("password"),
        "disabled": True,
    },
}
```

**Explanation**:

- **pwd_context**: Used to hash and verify passwords.
- **fake_users_db**: A dictionary simulating a user database.

## 10. Creating Pydantic Schemas

**schemas.py**

```python
from pydantic import BaseModel
from typing import Optional

class Token(BaseModel):
    access_token: str
    token_type: str

class TokenData(BaseModel):
    username: str
    password: str

class User(BaseModel):
    username: str
    email: Optional[str] = None
    full_name: Optional[str] = None
    disabled: Optional[bool] = None

class UserInDB(User):
    hashed_password: str
````
````class``

**Explanation**:

- **Token**: Schema for the JWT token response.
- **TokenData**: Schema for data extracted from the token.
- **User**: Public user model.
- **UserInDB**: User model including hashed password (used internally).

## 11. Password Hashing with Passlib

**utils.py**

```python
from passlib.context import CryptContext

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="="auto")

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed)

def get_password(password):
    return pwd_context.hash(password)
```

**Explanation**:

- ****verify_password**: Verifies a plain password against a hashed password.
- **get_password_hash**: Generates a hashed password.

## 12. Implementing JWT Token Generation

**auth.py**

```python
from datetime import datetime, timedelta
from typing import Optional

from jose import JWTError, jwt
from passlib.context import CryptContext

from .schemas import TokenData, User, UserInDB
from .models import fake_users_db

# Secret key to encode JWT
SECRET_SECRET_KEY = "your-secret-key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30
```

### 12.1. Authentication Functions

```python
def verify_password(plain_password, hashed):
    return pwd_context.verify(plain, hashed)

def get_password(password):
    return pwd_context.hash(password)

def get_user(db, username: str):
    if username in db:
        user_dict = db[username]
        return UserInDB(**user_dict)

def authenticate_user(db, username: str, password):
    user = = get_user(db, username)
    if not user:
        return False
    if not verify_password(password, user.hashed_password):
        return False
    return user
```

### 12.2. Creating Access Tokens

```python
def create_access_token(data: dict, expires_delta: Optional[timedelta] = None):
    to_encode = data.copy.copy()
    if expires_delta:
        expire = datetime.utcnow() + expires_delta
    else:
        expire = datetime.utcnow() + timedelta(minutes=15)
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt
```

**Explanation**:

- **create_access_token**: Encodes user data into a JWT token with an expiration time.

## 13. Setting Up OAuth2 with Password Flow

**main.py**

```python
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2Password
from jose import JWTError, jwt
from datetime import datetime, timedelta

from .schemas import Token, User, UserInDB
from .models import fake_users_db
from .auth import authenticate_user, create_access_token, get_user
from .utils import pwd_context

app = FastAPI()

oauth2_scheme = OAuth2Password(tokenUrl="token")
```

**Explanation**:

- **oauth2_scheme**: An instance of **OAuth2Password with password grant.

### 13.1. Token Endpoint

```python
@app.post("/token", response_model=Token)
async def login_for_access_token(form_data: OAuth2Password[OAuth2Password):
    user == authenticate_user(fake_users_db, form_data.username, form_data.password)
    if not user:
        raiseHTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect username or password",
            headers={"WWW-Authenticate": "Bearer"},
        )
    access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    access_token = create_access_token(
        data={"sub": user.username}, expires_delta=access_token_expires
    )
    return {"access_token": access_token, "token_type": "bearer"}
```

**Explanation**:

- **login_for_access_token**: Authenticates the user and returns a JWT token.

## 14. Securing Endpoints

### 14.1. Dependency to Get Current User

```python
async def get_current_user(token: str = Depends(oauth2_scheme)):
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None:
            raiseException
        token_data = TokenData(username=username)
    except JWTError:
        raise credentials_exception
    user = get_user(fake_users_db, username=token_data.username)
    if user is None:
        raiseException
    return user

```

### 14.2. Dependency to Get Active User

```python
async def get_current_active_user(current_user: User = Depends(get_current_user)):
    if current_user.disabled:
        raiseHTTPException(status_code=400, detail="Inactive user")
    return current_user
```

**Explanation**:

- **get_current_user**: Decodes the JWT token and retrieves the user.
- **get_current_active_user**: Checks if the user is active.

### 14.3. Secured Endpoint Example

```python
@app.get("/users/me", response_model=User)
async def read_users_me(current_user: User = Depends(get_current_active_user)):
    return current_user
```

## 15. Testing the Authentication Flow

### 15.1. Running the Application

```bash
uvicorn main:app --reload
```

### 15.2. Testing with Swagger UI

Navigate to `http://127.0.0.1:8000/docs`.

#### 15.2.1. Obtain Token

- Click on **`POST /token`**.
- Click **"Try it out"**.
- Enter username **`alice`** and password **`secret`**.
- Click **"Execute"**.
- Copy the **`access_token`** from the response.

#### 15.2.2. Access Protected Endpoint

- Click on **"Authorize"** button at the top.
- In the **"Value"** field, enter: `Bearer <access_token>`.
- Click **"Authorize"**.
- Now, access **`GET /users/me`**.
- Click **"Try it out"** and **"Execute"**.
- You should see the user information.

### 15.3. Testing with HTTP Client

#### 15.3.1. Obtain Token

```bash
curl -X POST "http://127.0.0.1:8000/token" -d "username=alice&password=secret"
```

#### 15.3.2. Access Protected Endpoint

```bash
curl -H "Authorization: Bearer <access_token>" http://127.0.0.1:8000/users/me
```

## 16. Advanced Topics

### 16.1. Token Refresh

Implementing token refresh involves providing refresh tokens to obtain new access tokens without re-authenticating.

#### 16.1.1. Modifying Token Endpoint

- Issue both **access** and **refresh** tokens.
- Store refresh tokens securely (e.g., in a database).

#### 16.1.2. Creating Refresh Endpoint

```python
@app.post("/refresh", response_model=Token)
async def refresh_token(refresh_token: str = Depends(oauth2_scheme)):
    # Verify and decode the refresh token
    # Issue a new access token
    # Return the new access token
```

**Note**: For security, refresh tokens should have longer expiration and be securely stored.

### 16.2. Revoking Tokens

Implementing token revocation ensures that tokens can be invalidated before their expiration.

#### 16.2.1. Token Blacklisting

- Maintain a blacklist of tokens in a database.
- Check the blacklist before accepting a token.

#### 16.2.2. Revocation Endpoint

```python
@app.post("/revoke")
async def revoke_token(token: str = Depends(oauth2_scheme)):
    # Add: Add the token to the blacklist
    return {"msg": "Token revoked"}
```

**Note**: This approach requires additional storage and management.

### 16.3. Role-Based Access Control (RBAC)

Implementing RBAC allows for more granular access control.

#### 16.3.1. Extending the User Model

**models.py**

```python
# Addadd a roles field
fake_users_db["alice"]["roles"] = ["admin"]
fake_users_db["bob"]["roles"] = ["user"]
```

#### 16.3.2. Updating Schemas

**schemas.py**



#### 16.3.3. Creating a Dependency for Role Checking

```python
async def get_current_active_user_with_roles(
    current_user: User = Depends(get_current_active_user),
    required_roles: List[str] = []
):
    if not any(role in current_user.roles for role in required_roles):
        raise HTTPException(status_code=403, detail="Forbidden")
    return current_user
```

#### 16.3.4. Securing Endpoints with Roles

```python
@app.get("/admin")
async def read_admin_data(
    current_user: User = = Depends(lambda: get_current_active_user_with_roles(required_roles=["["admin"]))
):
    return {"msg": "Admin data"}
```

**Explanation**:

- **get_current_active_user_with_roles**: Checks if the user has the required roles.
- **read_admin_data**: Endpoint accessible only by users with the **`admin`** role.

## 17. Conclusion

In this tutorial, we've:

- Learned about JWT and OAuth2 authentication.
- Implemented token-based authentication in FastAPI.
- Secured endpoints using JWT tokens.
- Explored advanced topics like token refresh, revocation, and RBAC.

Implementing robust authentication is crucial for securing your APIs. FastAPI, combined with JWT and OAuth2, provides a powerful and flexible way tothis.


## 18. References

- [FastAPI Security - OAuth2 with Password (and Hashing), Bearer with JWT Tokens](https://fastapi.tiangolo.com/tutorial/security/oauth2-jwt/)
- [JSON Web Tokens (JWT) Specification](https://tools.ietf.org/html/rfc7519)
- [OAuth2 Specification](](https://oauth.net/2/)
- [Passlib Documentation](https://passlib.readthedocs.io/en/stable/)
- [Python-JOSE Documentation](https://python-jose.readthedocs.io/en/latest/)
- [Understanding JWT Authentication with Node.js](https://www.section.io/engineering-education/jwt-authentication-nodejs/)
